In [1]:
!pip install transformers

In [4]:
import torch
from transformers import BertTokenizer,BertForQuestionAnswering
import textwrap

In [15]:
import json
import firebase_admin
from firebase_admin import credentials, db

In [12]:
firebase_initialization = False

In [13]:
def initialize(): #run only once-per iteration
  if(firebase_initialization == False):
    cred_obj = credentials.Certificate("/content/naanbert-firebase-adminsdk-k1t8m-65d91682f0.json")
    default_app = firebase_admin.initialize_app(cred_obj,{'databaseURL': 'https://naanbert-default-rtdb.firebaseio.com/'})

In [18]:
initialize()

In [22]:
def get_context(key):
  ref = db.reference('/')
  data = ref.get()
  abstract = data[key]
  return abstract
print(get_context('Introduction'))

Aravinda Eyecare System's vision is to eliminate needless blindness. The mission of Aravinda Eyecare System is to eliminate needless blindness by providing compassionate and quality eye care affordable to all. NABH stands for National Accreditation Board for Hospitals and Healthcare providers. It is a constituent board of the Quality Council of India (QCI) and was set up to establish and operate an accreditation programme for healthcare organizations in India. Initially, it was a voluntary programme but now includes mandated entry-level standards. NABH accredits all types of healthcare delivery organizations, including blood banks, imaging centers, AYUSH, nursing homes, clinics, and more. The NABH accreditation process involves three levels: Pre-accreditation entry level, Pre-accreditation progressive level, and Full accreditation. Healthcare organizations can be classified into two main categories: Small healthcare organizations (SHCO), which have less than 50 beds, and larger healthc

In [23]:
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
question = "How many parameters does BERT-large have?"
answer_text = "BERT-large is really big... it has 24-layers and an embedding size of 1,024, for a total of 340M parameters! Altogether it is 1.34GB, so expect it to take a couple minutes to download to your Colab instance."

In [8]:
input_ids = tokenizer.encode(question, answer_text)

In [9]:
def answer_question(question, answer_text):
    '''
    Takes a `question` string and an `answer_text` string (which contains the
    answer), and identifies the words within the `answer_text` that are the
    answer. Prints them out.
    '''
    # ======== Tokenize ========
    # Apply the tokenizer to the input text, treating them as a text-pair.
    input_ids = tokenizer.encode(question, answer_text)

    # Report how long the input sequence is.
    print('Query has {:,} tokens.\n'.format(len(input_ids)))

    # ======== Set Segment IDs ========
    # Search the input_ids for the first instance of the `[SEP]` token.
    sep_index = input_ids.index(tokenizer.sep_token_id)

    # The number of segment A tokens includes the [SEP] token istelf.
    num_seg_a = sep_index + 1

    # The remainder are segment B.
    num_seg_b = len(input_ids) - num_seg_a

    # Construct the list of 0s and 1s.
    segment_ids = [0]*num_seg_a + [1]*num_seg_b

    # There should be a segment_id for every input token.
    assert len(segment_ids) == len(input_ids)

    # ======== Evaluate ========
    # Run our example through the model.
    outputs = model(torch.tensor([input_ids]), # The tokens representing our input text.
                    token_type_ids=torch.tensor([segment_ids]), # The segment IDs to differentiate question from answer_text
                    return_dict=True)

    start_scores = outputs.start_logits
    end_scores = outputs.end_logits

    # ======== Reconstruct Answer ========
    # Find the tokens with the highest `start` and `end` scores.
    answer_start = torch.argmax(start_scores)
    answer_end = torch.argmax(end_scores)

    # Get the string versions of the input tokens.
    tokens = tokenizer.convert_ids_to_tokens(input_ids)

    # Start with the first token.
    answer = tokens[answer_start]

    # Select the remaining answer tokens and join them with whitespace.
    for i in range(answer_start + 1, answer_end + 1):

        # If it's a subword token, then recombine it with the previous token.
        if tokens[i][0:2] == '##':
            answer += tokens[i][2:]

        # Otherwise, add a space then the token.
        else:
            answer += ' ' + tokens[i]

    print('Answer: "' + answer + '"')

In [25]:
abstract = get_context("Introduction")
wrapper = textwrap.TextWrapper(width=80)

In [26]:
question = "organizations that NABH accredit"

answer_question(question,abstract)

Query has 313 tokens.

Answer: "nabh accredits all types of healthcare delivery organizations"
